In [ ]:
"""
    Code adapted from https://github.com/vishaldas/CNN_based_impedance_inversion
"""

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'
import numpy as np
from random import randint
from numpy import genfromtxt

In [ ]:
# Loading Impedance and seismic along well and checking if seismic using convolution matches the real seismic 

# Load seismic trace
seismic_data = genfromtxt('../data/volve/seismic_trace_15_9_F-15-A.csv', delimiter=';')

time_seismic = -np.round(seismic_data[:,0])
time_seismic, unique_indices = np.unique(time_seismic, return_index=True)
dt_seismic = time_seismic[1] - time_seismic[0]

seismic_trace = seismic_data[unique_indices,1]
Ip_trace = seismic_data[unique_indices,2]

start_time = 2340.0
end_time = 2500.0

seismic_trace_cropped = seismic_trace[np.where((time_seismic >= start_time) & (time_seismic <= end_time))]
Ip_trace_cropped = Ip_trace[np.where((time_seismic >= start_time) & (time_seismic <= end_time))]
time_cropped = time_seismic[np.where((time_seismic >= start_time) & (time_seismic <= end_time))]


# Load statistical source wavelet
source_statistical = genfromtxt('../data/volve/statistical_source_wavelet.txt', delimiter='	')
dt_source = 1e-3
plt.plot(source_statistical[:,1])
plt.show()


# Convolution of source with reflectivity series 
r0 = 0.5*np.diff(np.log(Ip_trace_cropped))
r0 = np.insert(r0,0, 0)
ss = np.convolve(r0, source_statistical[:,1], mode='same')
ss_normalized = (ss - np.min(ss))/(np.max(ss) - np.min(ss))

fig, axs = plt.subplots(1,3)
axs[0].plot(seismic_trace_cropped, time_cropped, '--k' )
axs[0].plot(1-ss_normalized, time_cropped, '-b' )
axs[1].plot(Ip_trace_cropped, time_cropped)
axs[2].plot(r0, time_cropped)
plt.show()


# Note to match 1-ss_normalized from impedance inversion

In [ ]:
# Load stochastically generated impedance 
Ip = np.loadtxt('../data/volve/Ip_exported.txt')
Ip_reshape = np.reshape(Ip, ((151,151,160)), order='F')

In [ ]:
# Trace by trace normalization and convolution 
ss_new = np.zeros(Ip_reshape.shape)
Ip_normalized = np.zeros(Ip_reshape.shape)
for i in range(Ip_reshape.shape[0]):
    for j in range(Ip_reshape.shape[1]):
        Ip_trace = Ip_reshape[i,j,:]
        Ip_normalized[i,j,:] = (Ip_trace - np.min(Ip_trace))/(np.max(Ip_trace) - np.min(Ip_trace))
        r0_trace = 0.5*np.diff(np.log(Ip_trace))
        r0_trace = np.insert(r0_trace,0, 0)
        ss_trace = np.convolve(r0_trace, source_statistical[:,1], mode='same')
        ss_new[i,j,:] = (ss_trace - np.min(ss_trace))/(np.max(ss_trace) - np.min(ss_trace))

In [ ]:
# Plot of results of convolution
time_new = np.arange(0,ss_new.shape[2]*1e-3, 1e-3)
nx = 100
ny = 100

plt.figure(num=None, figsize=(3, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(ss_new[nx,ny,:], time_new, '--r')
plt.plot(Ip_normalized[nx,ny,:], time_new, '--b')
plt.gca().invert_yaxis()

In [ ]:
# Creating training, validation and test data 

# Validation dataset created near the area of the well
indx_i_well_start = 20
indx_i_well_end = 45
indx_j_well_start = 32
indx_j_well_end = 54
valX = ss_new[indx_i_well_start:indx_i_well_end, indx_j_well_start:indx_j_well_end, :]
valX = valX.reshape(-1, valX.shape[-1])
valX = np.reshape(valX, (valX.shape[0], valX.shape[1], 1))
x_valid, x_test = np.split(valX, 2)

valIp = Ip_normalized[indx_i_well_start:indx_i_well_end, indx_j_well_start:indx_j_well_end, :]
valIp = valIp.reshape(-1, valIp.shape[-1])
valIp = np.reshape(valIp, (valIp.shape[0], valIp.shape[1], 1))
y_valid, y_test = np.split(valIp, 2)

# Flatten rest of indices 
ss_data_rest = ss_new[np.r_[0:(indx_i_well_start-1),(indx_i_well_end+1):ss_new.shape[0]], :, :]
ss_data_rest = ss_data_rest[:, np.r_[0:(indx_j_well_start-1),(indx_j_well_end+1):ss_new.shape[1]],:]
ss_data_rest = ss_data_rest.reshape(-1, ss_data_rest.shape[-1])

Ip_data_rest = Ip_normalized[np.r_[0:(indx_i_well_start-1),(indx_i_well_end+1):Ip_normalized.shape[0]], :, :]
Ip_data_rest = Ip_data_rest[:, np.r_[0:(indx_j_well_start-1),(indx_j_well_end+1):Ip_normalized.shape[1]],:]
Ip_data_rest = Ip_data_rest.reshape(-1, Ip_data_rest.shape[-1])

# Selecting randomly training dataset
howMany = 750 # Seismic data considered out of all data points 
np.random.seed(9) # For replication of results
indxRand = [randint(0,ss_data_rest.shape[0]-1) for p in range(0,howMany)]

trainX = (ss_data_rest[indxRand, :])
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

trainIp = (Ip_data_rest[indxRand, :])
trainIp = np.reshape(trainIp, (trainIp.shape[0], trainIp.shape[1], 1))

sampleNumber = 100
time = np.arange(0, trainX.shape[1]*1e-3, 1e-3)
fig, ax1 = plt.subplots()
ax1.plot(time, trainX[sampleNumber,:,0], 'b-')
ax1.set_xlabel('time (s)')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Seismic Amplitude', color='b')
ax1.tick_params('y', colors='b')

ax2 = ax1.twinx()
ax2.plot(time, trainIp[sampleNumber,:,0], 'r-')
ax2.set_ylabel('Impedance', color='r')
ax2.tick_params('y', colors='r')

fig.tight_layout()
plt.show()